In [1]:
#!pip install yfinance

In [2]:
#!pip install plotly

In [1]:
import warnings
warnings.filterwarnings('ignore')

import math
import polars as pl
import sklearn as sklearn
import scipy as scipy
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sys
import tensorflow.keras as keras
import tensorflow as tf
import cv2 as opencv
import numpy as np
import yfinance

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, ReLU, Add, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import plotly
from plotly.graph_objs import *
import plotly.tools as tls
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.express as px
import plotly.offline as py
import plotly.graph_objs as go
from sklearn.metrics import mean_squared_error, explained_variance_score
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

2024-11-14 20:58:58.801578: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

GPU is available


2024-11-14 20:59:01.002687: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-14 20:59:01.025192: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-14 20:59:01.025313: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
def get_yf_hist(symbol_list,startDate,endDate,interval):
  import matplotlib.pyplot as plt
  import datetime
  import yfinance as yf

  # Get historical pricing data
  data = yf.download(symbol_list, startDate, endDate, interval)

  return data


def get_intraday(symbol_list,period,interval):
  intraday_list = []
  intraday_df = pd.DataFrame()
  for sym in symbol_list:
    intraday = yf.download(tickers=sym,
                           period=period,
                           interval=interval)
    intraday['symbol'] = sym
    intraday_df = pd.concat([intraday_df,intraday])

  return intraday_df

def rolling_zscore(data,return_period,window_length):
  log_returns = (np.log(data / data.shift(return_period)))
  zscore = (log_returns - log_returns.rolling(window_length).mean() / log_returns.rolling(window_length).std())
  #results_dict = dict({'log_returns':log_returns})
  results_df = pd.DataFrame(zscore)

  return results_df


  # convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

def rmse(y_true,y_pred):
    
    # Ensure both lists have the same length
    if len(y_true) != len(y_pred):
        raise ValueError("The length of actual and predicted values must be the same")
    
    # Calculate the squared differences
    squared_diff = [(yt-yp) ** 2 for yt, yp in zip(y_true,y_pred)]
    
    # Calculate mean of the squared differences
    mean_squared_diff = sum(squared_diff) / len(y_true)
    
    # Return the square root of the mean squared difference
    rmse = math.sqrt(mean_squared_diff)
    
    return rmse

In [46]:
#symbol_list = ['GS','JPM','C','WFC','STT']
#symbol_list = ['^GSPC','AMD','NVDA','TXN']
#symbol_list = ['^GSPC','SPY','MSFT','AAPL','NVDA','AMZN','LLY','JPM','AMD','XOM','UNH','PG','JNJ','HD','MRK','COST','CVX','NFLX','CRM','WMT','AMD','BAC','PEP','KO','LIN','ADBE','WFC','TMO','DIS','ACN','MCD','CSCO','ORCL','ABT','CAT','QCOM','INTU','AMGN','BKNG','LOW','ISRG','CRM']

#symbol_list = ['MA','V','^GSPC']
symbol_list = ['^GSPC','UNH','GS','^DJI','MSFT','HD','SHW','CAT','CRM','V','AMGN','MCD','AXP','TRV','JPM','HON','AAPL','AMZN','IBM','PG','CVX','JNJ','NVDA','BA','MMM','DIS','MRK','WMT','NKE','KO','CSCO','VZ']

startDate = '1900-01-01'
endDate = '2024-11-14'

close_df = get_yf_hist(symbol_list,startDate,endDate,interval='1d')

close_df = close_df['Adj Close']


def structure_data(symbol_list,close_df):
    if len(symbol_list) > 1:
        special_col = '^DJI'
        cols = [special_col] + [col for col in close_df.columns if col != special_col]
        close_df = close_df[cols]
        return close_df
    else:
        return close_df

close_df = structure_data(symbol_list,close_df)
    
close_df.head()

[*********************100%***********************]  32 of 32 completed


Ticker,^DJI,AAPL,AMGN,AMZN,AXP,BA,CAT,CRM,CSCO,CVX,...,NKE,NVDA,PG,SHW,TRV,UNH,V,VZ,WMT,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1927-12-30 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.660000
1928-01-03 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.760000
1928-01-04 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.719999
1928-01-05 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.549999
1928-01-06 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.660000


In [47]:
close_corrs_df = close_df.corr()
close_corrs_diffs_df = close_df.diff()
close_corrs_diffs_df = close_corrs_diffs_df.corr()

import plotly.figure_factory as ff

fig = ff.create_annotated_heatmap(z=close_corrs_df.values,
                                 x=close_corrs_df.columns.tolist(),
                                 y=close_corrs_df.index.tolist(),
                                 showscale=True,
                                 colorscale='Viridis')
fig.show()

In [150]:
# This code helps you visualize how many null values are in each column
print(f"NULL value counts: {pd.isnull(close_df).sum().sort_values(ascending=False)}")
print("")
print(f"NULL Value percentages: {(pd.isnull(close_df).sum()).sort_values(ascending=False) / len(close_df)*100}")

df2 = close_df
df2 = df2.dropna(axis=0)
df2 = df2.sort_index(ascending=True)

print(f'df2 shape: {df2.shape}')
print(f'df2 head: {df2.head()}')

df2_values = df2.values
df2_values = df2_values.astype('float32')

#n_out variable is the number of periods to forecast outward in time
#n_in variable is the number of periods to lag the time series of each variable

df2_fmt = series_to_supervised(df2_values,1,1)

f"df2_values shape: {df2_values.shape}"

df2_fmt2 = df2_fmt.iloc[:,:-len(df2.columns)+1]

print(f"df2_fmt shape: {df2_fmt2.shape}")
print(f"df2_fmt2 head: {df2_fmt2.head()}")

n_in = 1

scaler = MinMaxScaler(feature_range=(0,1))
scaled = scaler.fit_transform(df2_values)
df2_reframed = series_to_supervised(scaled,n_in,1)
df2_reframed3 = series_to_supervised(scaled,n_in,1)

df2_reframed = df2_reframed.iloc[:,:-len(df2.columns)]
df2_reframed['var1(t)'] = df2_reframed3['var1(t)']

print(f"df2_reframed head {df2_reframed.head()}")
# Now we have the primary stock we want to prdict (var1), its prices shifted back by 1 unit (because that was our chosen input value for our supervised learning function), as well as the other stocks that are also shifted all in the same dataframe

values = df2_reframed.values

train_split = round(int(len(values)*0.60))
test_split = (len(values) - train_split)

train = values[:train_split]
test = values[train_split:]

X_train,y_train = train[:,:-1],train[:,-1]
X_test,y_test = test[:,:-1],test[:,-1]

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}, X_test shape: {X_test.shape},y_test shape: {y_test.shape}")

# reshape input to be 3D [samples, timesteps, features]
X_train = X_train.reshape(X_train.shape[0],1,X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0],1,X_test.shape[1])

X_train.shape
X_test.shape

df = close_df

NULL value counts: Ticker
V        20141
CRM      19200
GS       17909
NVDA     17839
AMZN     17414
^DJI     16056
CSCO     15583
MSFT     14588
UNH      14235
VZ       14006
AMGN     13897
HD       13457
AAPL     13262
NKE      13254
SHW      13074
JPM      13074
TRV      11981
WMT      11168
AXP      11108
MCD       9644
JNJ       8509
KO        8509
MMM       8509
MRK       8509
IBM       8509
HON       8509
DIS       8509
PG        8509
CVX       8509
CAT       8509
BA        8509
^GSPC        0
dtype: int64

NULL Value percentages: Ticker
V        82.765564
CRM      78.898706
GS       73.593589
NVDA     73.305938
AMZN     71.559482
^DJI     65.979043
CSCO     64.035340
MSFT     59.946579
UNH      58.495993
VZ       57.554962
AMGN     57.107047
HD       55.298952
AAPL     54.497637
NKE      54.464763
SHW      53.725087
JPM      53.725087
TRV      49.233614
WMT      45.892747
AXP      45.646189
MCD      39.630162
JNJ      34.966098
KO       34.966098
MMM      34.966098
MRK      34.

In [151]:
# Parameters
patience = 10
sequence_length = 90  # Number of days to look back
features = df.shape[1]  # Number of features
epochs = 50
batch_size = 16
# The below filter size seems to have the best results thus far
#filter1 = 128

# This is our new experimental filter value
#filter1 = 64
filter1 = 164

early_stopping = EarlyStopping(patience=patience,monitor='val_loss',restore_best_weights=True)
checkpoint_filepath = "/kaggle/working Checkpoints/best_model{epoch:02d}_{val_loss:.2f}.keras"
model_checkpoint = ModelCheckpoint(filepath=checkpoint_filepath,monitor='val_loss',verbose=1,save_best_only=True,mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience=(patience/2),verbose=1,mode='auto',min_delta=0.0001,cooldown=0,min_lr=0.0)

# df = close_df

# # Normalize data
# scaler = MinMaxScaler()
# data = scaler.fit_transform(df)

# # Prepare sequences and targets
# X, y = [], []
# for i in range(sequence_length, len(data)):
#     X.append(data[i-sequence_length:i])
#     y.append(data[i][0])  # SPY's price as target

# X, y = np.array(X), np.array(y)

# # Train/test split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)


# TCN model architecture
def build_tcn(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv1D(filter1, kernel_size=5, dilation_rate=1, padding="causal")(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    # Stacking TCN blocks with residual connections
    for dilation_rate in [2, 4, 6, 8, 16, 64]:
        previous_x = x
        x = Conv1D(filter1, kernel_size=5, dilation_rate=dilation_rate, padding="causal")(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = Add()([x, previous_x])  # Residual connection
        x = Dropout(rate=0.3)(x)
    
    x = Flatten()(x)
    outputs = Dense(1)(x)
    
    model = Model(inputs, outputs)
    return model

# Compile model
input_shape = (X_train.shape[1], X_train.shape[2])
model = build_tcn(input_shape)
model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')

# Train model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size,callbacks=[early_stopping,reduce_lr])

# Evaluate model
mse = model.evaluate(X_test, y_test)
print("Test MSE:", mse)

predictions = model.predict(X_test)

Epoch 1/50
158/158 [==============================] - 6s 24ms/step - loss: 16.0785 - val_loss: 0.0906 - lr: 0.0100
Epoch 2/50
158/158 [==============================] - 3s 19ms/step - loss: 0.3211 - val_loss: 0.0691 - lr: 0.0100
Epoch 3/50
158/158 [==============================] - 3s 19ms/step - loss: 0.0226 - val_loss: 0.0771 - lr: 0.0100
Epoch 4/50
158/158 [==============================] - 3s 21ms/step - loss: 0.0123 - val_loss: 0.1231 - lr: 0.0100
Epoch 5/50
158/158 [==============================] - 3s 21ms/step - loss: 0.0176 - val_loss: 0.0743 - lr: 0.0100
Epoch 6/50
158/158 [==============================] - 3s 20ms/step - loss: 0.0294 - val_loss: 0.4237 - lr: 0.0100
Epoch 7/50
158/158 [==============================] - 3s 22ms/step - loss: 0.0291 - val_loss: 0.0050 - lr: 0.0100
Epoch 8/50
158/158 [==============================] - 3s 21ms/step - loss: 0.0660 - val_loss: 1.0714 - lr: 0.0100
Epoch 9/50
158/158 [==============================] - 3s 20ms/step - loss: 0.2463 - val

In [152]:
trace_0 = go.Scatter(y=history.history['loss'],
                     name='train',
                     mode='lines',
                     marker= {'color':'#FF6F61','opacity':0.9})

trace_1 = go.Scatter(y=history.history['val_loss'],
                     name='test',
                     mode='lines',
                     marker= {'color':'#1d5dec','opacity':0.9})

data = [trace_0,trace_1]

layout = go.Layout(title="train/test logarithmic loss comparison",
                  template='plotly_dark',
                   height=600,
                   width=1000)

fig = go.Figure(data,layout=layout)
fig.show()

In [153]:
# Deploy the model, return predictions, and reformat the X_test data
predicted = model.predict(X_test)
X_test_rs = X_test.reshape(X_test.shape[0],X_test.shape[2])

# Rescale the test data
new_df = pd.DataFrame(X_test_rs[:,1:])
new_df2 = pd.concat([pd.DataFrame(y_test),pd.DataFrame(new_df)],axis=1)
new_df2.columns = df2.columns
new_df3 = scaler.inverse_transform(new_df2)
new_df3 = pd.DataFrame(new_df3,columns=df2.columns)

# Rescale the predictions data
new_pred_df = pd.DataFrame(predicted)
new_pred = pd.concat([new_pred_df,pd.DataFrame(new_df)],axis=1)
new_pred = scaler.inverse_transform(new_pred)
new_pred = pd.DataFrame(new_pred,columns=df2.columns)

# Create a DataFrame with the predicted and actual values and take the difference between the two
results_df = pd.concat([new_pred.iloc[:,0],new_df3.iloc[:,0]],axis=1)
results_df.columns = ['theta_hat','theta']
results_df['Difference'] = results_df['theta_hat'] - results_df['theta']

print(f"Root Mean Squared Error: {rmse(results_df['theta_hat'],results_df['theta'])}")

results_df

53/53 [==============================] - 0s 5ms/step
Root Mean Squared Error: 1487.5039626531498


,theta_hat,theta,Difference
0,22183.113281,24946.509766,-2763.396484
1,22218.970703,24610.908203,-2391.937500
2,21870.691406,24727.269531,-2856.578125
3,21955.792969,24682.310547,-2726.517578
4,21908.898438,23957.890625,-2048.992188
...,...,...,...
1673,46078.886719,43729.339844,2349.546875
1674,46110.207031,43988.988281,2121.218750
1675,46390.554688,44293.128906,2097.425781
1676,46601.957031,43910.980469,2690.976562


In [149]:
trace_0 = go.Scatter(y=results_df['theta_hat'],
                     name='predicted',
                     mode='lines',
                     marker= {'color':'#FF6F61','opacity':0.9})

trace_1 = go.Scatter(y=results_df['theta'],
                     name=df2.columns[0],
                     mode='lines',
                     marker= {'color':'#1d5dec','opacity':0.9})

data = [trace_0,trace_1]


layout = go.Layout(title=f"LSTM model predictions vs. actual price of {df2.columns[0]}",
                  template='plotly_dark',
                   height=900,
                   width=1500)

fig = go.Figure(data,layout=layout)
fig.show()

In [76]:
# def reset_model(model):
#   del model
#   model = Sequential()
#   return model

# model = reset_model(model)
print(explained_variance_score(y_true=results_df['theta'],y_pred=results_df['theta_hat']))

0.664752721786499


The below model has multiple TCN layers instead of only one like the above model

In [27]:
# Parameters
patience = 30
sequence_length = 90  # Number of days to look back
features = df.shape[1]  # Number of features
epochs = 100
batch_size = 16
# The below filter size seems to have the best results thus far
#filter1 = 128

# This is our new experimental filter value
filter1 = 10


early_stopping = EarlyStopping(patience=patience,monitor='val_loss',restore_best_weights=True)
checkpoint_filepath = "/kaggle/working Checkpoints/best_model{epoch:02d}_{val_loss:.2f}.keras"
model_checkpoint = ModelCheckpoint(filepath=checkpoint_filepath,monitor='val_loss',verbose=1,save_best_only=True,mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience=(patience/2),verbose=1,mode='auto',min_delta=0.0001,cooldown=0,min_lr=0.0)



def build_tcn(input_shape, num_layers_per_block=2):
    inputs = Input(shape=input_shape)
    
    # Initial Conv1D layer
    x = Conv1D(filter1, kernel_size=2, dilation_rate=1, padding="causal")(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    # Stacked TCN blocks with multiple Conv1D layers
    for dilation_rate in [2, 4, 8, 16]:
        previous_x = x
        for _ in range(num_layers_per_block):  # Stack `num_layers_per_block` Conv1D layers
            x = Conv1D(filter1, kernel_size=2, dilation_rate=dilation_rate, padding="causal")(x)
            x = BatchNormalization()(x)
            x = ReLU()(x)
        
        # Add a residual connection to the block's input
        x = Add()([x, previous_x])
    
    x = Flatten()(x)
    outputs = Dense(1)(x)
    
    model = Model(inputs, outputs)
    return model

# Compile and use the model as before
model = build_tcn(input_shape)
model.compile(optimizer=Adam(), loss='mean_squared_error')
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size,callbacks=[early_stopping,reduce_lr])


Epoch 1/100
131/131 [==============================] - 6s 25ms/step - loss: 0.2966 - val_loss: 0.1800 - lr: 0.0010
Epoch 2/100
131/131 [==============================] - 3s 24ms/step - loss: 0.0584 - val_loss: 0.6400 - lr: 0.0010
Epoch 3/100
131/131 [==============================] - 3s 24ms/step - loss: 0.0366 - val_loss: 2.8825 - lr: 0.0010
Epoch 4/100
131/131 [==============================] - 3s 24ms/step - loss: 0.0256 - val_loss: 8.7736 - lr: 0.0010
Epoch 5/100
131/131 [==============================] - 3s 24ms/step - loss: 0.0199 - val_loss: 0.7688 - lr: 0.0010
Epoch 6/100
131/131 [==============================] - 3s 24ms/step - loss: 0.0147 - val_loss: 1.8242 - lr: 0.0010
Epoch 7/100
131/131 [==============================] - 3s 24ms/step - loss: 0.0119 - val_loss: 0.1198 - lr: 0.0010
Epoch 8/100
131/131 [==============================] - 3s 25ms/step - loss: 0.0097 - val_loss: 0.2121 - lr: 0.0010
Epoch 9/100
131/131 [==============================] - 3s 25ms/step - loss: 0.00

In [28]:
trace_0 = go.Scatter(y=history.history['loss'],
                     name='train',
                     mode='lines',
                     marker= {'color':'#FF6F61','opacity':0.9})

trace_1 = go.Scatter(y=history.history['val_loss'],
                     name='test',
                     mode='lines',
                     marker= {'color':'#1d5dec','opacity':0.9})

data = [trace_0,trace_1]

layout = go.Layout(title="train/test logarithmic loss comparison",
                  template='plotly_dark',
                   height=600,
                   width=1000)

fig = go.Figure(data,layout=layout)
fig.show()

In [29]:
# Deploy the model, return predictions, and reformat the X_test data
predicted = model.predict(X_test)
X_test_rs = X_test.reshape(X_test.shape[0],X_test.shape[2])

# Rescale the test data
new_df = pd.DataFrame(X_test_rs[:,1:])
new_df2 = pd.concat([pd.DataFrame(y_test),pd.DataFrame(new_df)],axis=1)
new_df2.columns = df2.columns
new_df3 = scaler.inverse_transform(new_df2)
new_df3 = pd.DataFrame(new_df3,columns=df2.columns)

# Rescale the predictions data
new_pred_df = pd.DataFrame(predicted)
new_pred = pd.concat([new_pred_df,pd.DataFrame(new_df)],axis=1)
new_pred = scaler.inverse_transform(new_pred)
new_pred = pd.DataFrame(new_pred,columns=df2.columns)

# Create a DataFrame with the predicted and actual values and take the difference between the two
results_df = pd.concat([new_pred.iloc[:,0],new_df3.iloc[:,0]],axis=1)
results_df.columns = ['theta_hat','theta']
results_df['Difference'] = results_df['theta_hat'] - results_df['theta']

print(f"Root Mean Squared Error: {rmse(results_df['theta_hat'],results_df['theta'])}")

results_df

66/66 [==============================] - 0s 5ms/step
Root Mean Squared Error: 55.81160885732388


,theta_hat,theta,Difference
0,86.689781,82.853264,3.836517
1,85.242088,83.788368,1.453720
2,86.539658,84.504028,2.035629
3,88.848427,84.685341,4.163086
4,89.264931,85.878082,3.386848
...,...,...,...
2084,632.880554,514.070007,118.810547
2085,618.866272,513.760010,105.106262
2086,609.295288,516.340027,92.955261
2087,615.517883,515.010010,100.507874


In [30]:
trace_0 = go.Scatter(y=results_df['theta_hat'],
                     name='predicted',
                     mode='lines',
                     marker= {'color':'#FF6F61','opacity':0.9})

trace_1 = go.Scatter(y=results_df['theta'],
                     name=df2.columns[0],
                     mode='lines',
                     marker= {'color':'#1d5dec','opacity':0.9})

data = [trace_0,trace_1]


layout = go.Layout(title=f"LSTM model predictions vs. actual price of {df2.columns[0]}",
                  template='plotly_dark',
                   height=900,
                   width=1500)

fig = go.Figure(data,layout=layout)
fig.show()